# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 

        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra code

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    ''')

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create, populate, and query tables

### Queries the tables are modeled for:
##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* * *
#### First query: artist_song_length_by_session_and_item

The columns **sessionId** and **itemInSession** were chosen as PRIMARY KEY combination because the queries will be filtered by these two columns.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS artist_song_length_by_session_and_item (
            sessionId int,
            itemInSession int,
            artist_name text,
            song_title text,
            song_length float,
            PRIMARY KEY(sessionId, itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = """INSERT INTO artist_song_length_by_session_and_item (
                    sessionId,
                    itemInSession,
                    artist_name, 
                    song_title,
                    song_length
                    )
                    VALUES (%s, %s, %s, %s, %s);
                """
        artist_name, first_name, gender, itemInSession, last_name, song_length, level, location, sessionId, song_title, userId = line

        session.execute(query, ( int(sessionId), int(itemInSession), artist_name, song_title, float(song_length) ) )

##### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
query = """select artist_name, song_title, song_length 
            from artist_song_length_by_session_and_item 
            WHERE 
                sessionId = 338 AND 
                itemInSession = 4
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

#### Second query: artist_song_user_by_userid_and_sessionid

The columns **userId** and **sessionId** were defined as a PRIMARY KEY as queries will filter by them. **itemInSession** was added as clustering column because the queries are expected to sort the output by it.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS artist_song_user_by_userid_and_sessionid (
                userId int,
                sessionId int,
                itemInSession int,
                artist_name text,
                song_title text,
                first_name text,
                last_name text,
                PRIMARY KEY((userId, sessionId), itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = """INSERT INTO artist_song_user_by_userid_and_sessionid (
                    userId,
                    sessionId,
                    itemInSession,
                    artist_name,
                    song_title,
                    first_name,
                    last_name) 
                    VALUES (%s,%s,%s,%s,%s,%s,%s)
                """
        
        artist_name, first_name, gender, itemInSession, last_name, song_length, level, location, sessionId, song_title, userId = line
        
        session.execute(query, ( int(userId), int(sessionId), int(itemInSession), artist_name, song_title, first_name, last_name ))

In [ ]:
query = """select artist_name, song_title, first_name, last_name 
            from artist_song_user_by_userid_and_sessionid
            WHERE 
                userId = 10 AND 
                sessionId = 182;
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

#### Third query: user_by_songtitle

The columns **song_title** and **userId** were selected as PRIMARY KEY because the queries will filter by **song_title** and there might be more than one **userId** related to it.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS user_by_songtitle (
                song_title text,
                userId int,
                first_name text,
                last_name text,
                PRIMARY KEY(song_title, userId))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = """INSERT INTO user_by_songtitle (
                    song_title,
                    userId,
                    first_name,
                    last_name) 
                    VALUES (%s,%s,%s, %s)
                """
        
        artist_name, first_name, gender, itemInSession, last_name, song_length, level, location, sessionId, song_title, userId = line
        
        session.execute(query, ( song_title, int(userId), first_name, last_name ))

In [ ]:
query = """select first_name, last_name from user_by_songtitle
            WHERE 
                song_title = 'All Hands Against His Own';
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print(row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [ ]:
query = "drop table artist_song_length_by_session_and_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_song_user_by_userid_and_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_by_songtitle"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()